In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# **Worklfow Management and meta job scheduler**

This tutorial illustrates how computations can be executed on large 
(or small) compute servers, aka.
high-performance-computers (HPC), aka. compute clusters, aka. supercomputers, etc.

This means, the computation is not executed on the local workstation 
(or laptop) but on some other computer.
This approach is particulary handy for large computations, which run
for multiple hours or days, since a user can 
e.g. shutdown or restart his personal computer without killing the compute job. 

*BoSSS* features a set of classes and routines 
(an API, application programing interface) for communication with 
compute clusters. This is especially handy for **scripting**,
e.g. for parameter studies, where dozens of computations 
have to be started and monitored.


# 1 Batch Processing

First, we have to select a *batch system* that we want to use.
Batch systems are a common approach to organize workloads (aka. compute jobs)
on compute clusters.
On such systems, a user typically does *not* starts a simulation manually.
Instead, he specifies a so-called *compute job*. The *scheduler* 
(aka. *batch system*) collects 
compute jobs from all users on the compute cluster, sorts them according to 
some priority and puts the jobs into some queue, also called *batch*.
The jobs in the batch are then executed in order, depending on the 
available hardware and the scheduling policies of the system.

The *BoSSS* API provides front-ends (clients) for the following 
batch system software:

- **BoSSS.Application.BoSSSpad.SlurmClient** for the 
Slurm Workload Manager (very prominent on Linux HPC systems)
- **BoSSS.Application.BoSSSpad.MsHPC2012Client**
for the Microsoft HPC Pack 2012 and higher
- **BoSSS.Application.BoSSSpad.MiniBatchProcessorClient** for the 
mini batch processor, a minimalistic, *BoSSS*-internal batch system which mimiks 
a supercomputer batch system on the local machine.


A list of clients for various batch systems, which are loaded at the 
**restart** command can be configured through the  
*\tt $\sim$/.BoSSS/etc/BatchProcessorConfig.json*-file.
If this file is missing, a default setting, containing a 
mini batch processor, is initialized. 

The list of all execution queues can be accessed through:




In [ ]:
ExecutionQueues

For the sake of simplicuity, we pick the first one:

In [ ]:

var myBatch = ExecutionQueues[0];

Note on the Mini Batch Processor:
The batch processor for local jobs can be started separately (by launching
**\tt MiniBatchProcessor.exe**), or from the worksheet;
In the latter case, it depends on the operating system, whether the 
\newline **\tt MiniBatchProcessor.exe** is terminated with the worksheet, 
or not.
If no mini-batch-processor is running, it is started 
at the initialization of the workflow management.

```csharp
MiniBatchProcessor.Server.StartIfNotRunning(false);
```


# 2 Initializing the workflow management

In order to use the workflow management, 
the very first thing we have to do is to initialize it by defineing 
a project name. 
This is used to generate names for the compute jobs and to 
identify sessions in the database:

In [ ]:
BoSSSshell.WorkflowMgm.Init("MetaJobManager_Tutorial");

We verify that we have no jobs defined so far ...

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs

BoSSScmdSilent BoSSSexeSilent

In [ ]:
NUnit.Framework.Assert.IsTrue(BoSSSshell.WorkflowMgm.AllJobs.Count == 0, "MetaJobManager tutorial: expecting 0 jobs on entry.");

and create, resp. open a *BoSSS* database on a local drive:

In [ ]:
var myLocalDb = CreateTempDatabase();

To specify the exact directory of the database, you might use
**OpenOrCreateDatabase**.

BoSSScmdSilent BoSSSexeSilent

In [ ]:
databases

# Loading a BoSSS-Solver and Setting up a Simulation

As an example, we use the workflow management tools to simulate 
incompressible channel flow, therefore we have to import the namespace,
and repeat the steps from the IBM example (Tutorial 2) in order to setup the
control object:

In [ ]:
using BoSSS.Application.XNSE_Solver;

We create a grid with boundary conditions:

In [ ]:
var xNodes       = GenericBlas.Linspace(0, 10 , 41); 
var yNodes       = GenericBlas.Linspace(-1, 1, 9); 
GridCommons grid = Grid2D.Cartesian2DGrid(xNodes, yNodes);
grid.DefineEdgeTags(delegate (double[] X) { 
    double x = X[0];
    double y = X[1]; 
    if (Math.Abs(y - (-1)) <= 1.0e-8) 
        return "wall"; // lower wall
    if (Math.Abs(y - (+1)) <= 1.0e-8) 
        return "wall"; // upper wall
    if (Math.Abs(x - (0.0)) <= 1.0e-8) 
        return "Velocity_Inlet"; // inlet
    if (Math.Abs(x - (+10.0)) <= 1.0e-8) 
        return "Pressure_Outlet"; // outlet
    throw new ArgumentOutOfRangeException("unknown domain"); 
});

And save it to the database:

In [ ]:
myLocalDb.SaveGrid(ref grid);

Next, we create the control object for the incompressible simulation:

control object instantiation

In [ ]:
var c = new XNSE_Control();
// general description:
int k                = 1;
string desc          = "Steady state, channel, k" + k; 
c.SessionName        = "SteadyStateChannel"; 
c.ProjectDescription = desc;
c.Tags.Add("k"+k);
// setting the grid:
c.SetGrid(grid);
// DG polynomial degree
c.SetDGdegree(k);
// Physical parameters:
double reynolds            = 20; 
c.PhysicalParameters.rho_A = 1; 
c.PhysicalParameters.mu_A  = 1.0/reynolds;
// Timestepping properties:
c.TimesteppingMode = AppControl._TimesteppingMode.Steady;

The specification of boundary conditions and initial values
is a bit more complicated if the job manager is used:

Since the solver is executed in an external program, the control object 
has to be saved in a file. For lots of complicated objects,
especially for delegates, C\# does not support serialization 
(converting the object into a form that can be saved on disk, or 
transmitted over a network), so a workaround is needed.
This is achieved e.g. by the **Formula** object, where a C\#-formula
is saved as a string.

In [ ]:
var WallVelocity = new Formula("X => 0.0", false); // 2nd Argument=false says that its a time-indep. formula.

Testing the formula:

In [ ]:
WallVelocity.Evaluate(new[]{0.0, 0.0}, 0.0) // evaluationg at (0,0), at time 0

In [ ]:
// [Deprecated]
/// A disadvantage of string-formulas is that they look a bit ``alien''
/// within the worksheet; therefore, there is also a little hack which allows 
/// the conversion of a static memeber function of a static class into a 
/// \code{Formula} object:

In [ ]:
// Deprecated, this option is no longer supported in .NET5
static class StaticFormulas {
    public static double VelX_Inlet(double[] X) {
        //double x  = X[0];
        double y  = X[0];
        double UX = 1.0 - y*y;
        return UX;
    }  
 
    public static double VelY_Inlet(double[] X) {
        return 0.0;
    }      
}

In [ ]:
// InletVelocityX = GetFormulaObject(StaticFormulas.VelX_Inlet);
//var InletVelocityY = GetFormulaObject(StaticFormulas.VelY_Inlet);

In [ ]:
var InletVelocityX = new Formula("X => 1 - X[0]*X[0]", false);
var InletVelocityY = new Formula("X => 0.0", false);

Finally, we set boundary values for our simulation. The initial values
are set to zero per default; for the steady-state simulation initial
values are irrelevant anyway:

Initial Values are set to 0

In [ ]:
c.BoundaryValues.Clear(); 
c.AddBoundaryValue("wall", "VelocityX", WallVelocity); 
c.AddBoundaryValue("Velocity_Inlet", "VelocityX", InletVelocityX);  
c.AddBoundaryValue("Velocity_Inlet", "VelocityY", InletVelocityY); 
c.AddBoundaryValue("Pressure_Outlet");

# 4 Activation and Monitoring of the the Job

Finally, we are ready to deploy the job at the batch processor;
In a usual work flow scenario, we **do not** want to (re-) submit the 
job every time we run the worksheet -- usually, one wants to run a job once.

The concept to overcome this problem is job activation. If a job is 
activated, the meta job manager first checks the databases and the batch 
system, if a job with the respective name and project name is already 
submitted. Only if there is no information that the job was ever submitted
or started anywhere, the job is submitted to the respective batch system.

First, a **Job**-object is created from the control object:

In [ ]:
var JobLocal = c.CreateJob();

This job is not activated yet, it can still be configured:

In [ ]:
JobLocal.Status

In [ ]:

NUnit.Framework.Assert.IsTrue(JobLocal.Status == JobStatus.PreActivation);

One can change e.g. the number of MPI processes:

In [ ]:
JobLocal.NumberOfMPIProcs = 1;

Note that these jobs are desigend to be **persistent**:
This means the computation is only started 
**once for a given control object**, no matter how often the worksheet
is executed. 

Such a behaviour is useful for expensive simulations, which run on HPC
servers over days or even weeks. The user (you) can close the worksheet
and maybe open and execute it a few days later, and he can access
the original job which he submitted a few days ago (maybe it is finished
now).

Then, the job is activated, resp. submitted, resp. deployed 
to one batch system.
If job persistency is not wanted, traces of the job can be removed 
on request during activation, causing a fresh job deployment at the
batch system:

In [ ]:
bool DeleteOldDeploymentsAndSessions = true; // use with care! Normally, you 
                                             // dont want this!!!
JobLocal.Activate(myBatch,  // execute the job in 'myBatch'
                  DeleteOldDeploymentsAndSessions); // causes fresh deployment

All jobs can be listed using the workflow management:

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs

Check the present job status:

In [ ]:
JobLocal.Status

In [ ]:
/// BoSSScmdSilent BoSSSexeSilent
NUnit.Framework.Assert.IsTrue(
   JobLocal.Status == JobStatus.PendingInExecutionQueue
   || JobLocal.Status == JobStatus.InProgress
   || JobLocal.Status == JobStatus.FinishedSuccessful);

Here, we block until both of our jobs have finished:

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(1000);

We examine the output and error stream of the job:
This directly accesses the *\tt stdout*-redirection of the respective job
manager, which may contain a bit more information than the 
`Stdout`-copy in the session directory.

In [ ]:
JobLocal.Stdout

Additionally we display the error stream and hope that it is empty:

In [ ]:
JobLocal.Stderr

We can also obtain the session 
which was stored during the execution of the job:

In [ ]:
var Sloc = JobLocal.LatestSession;
Sloc

 We can also list all attempts to run the job at the assigend processor:

In [ ]:
JobLocal.AllDeployments

In [ ]:
NUnit.Framework.Assert.IsTrue(JobLocal.AllDeployments.Count == 1, "MetaJobManager tutorial: Found more than one deployment.");

Finally, we check the status of our jobs:

In [ ]:
JobLocal.Status

 If anything failed, hints on the reason why are provides by the 
**GetStatus** method:

In [ ]:
JobLocal.GetStatus(WriteHints:true)

In [ ]:
NUnit.Framework.Assert.IsTrue(JobLocal.Status == JobStatus.FinishedSuccessful, "MetaJobManager tutorial: Job was not successful.");